In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set random seed for reproducible results
np.random.seed(42)
random.seed(42)

# Define expense categories and typical amounts
expense_categories = {
    'Groceries': {'min': 20, 'max': 150, 'frequency': 0.25},  # 25% of transactions
    'Restaurants': {'min': 15, 'max': 80, 'frequency': 0.15},
    'Gas': {'min': 30, 'max': 70, 'frequency': 0.08},
    'Shopping': {'min': 25, 'max': 200, 'frequency': 0.12},
    'Entertainment': {'min': 10, 'max': 100, 'frequency': 0.10},
    'Bills_Utilities': {'min': 50, 'max': 300, 'frequency': 0.08},
    'Transportation': {'min': 5, 'max': 50, 'frequency': 0.06},
    'Healthcare': {'min': 20, 'max': 250, 'frequency': 0.04},
    'Personal_Care': {'min': 10, 'max': 80, 'frequency': 0.05},
    'Miscellaneous': {'min': 5, 'max': 100, 'frequency': 0.07}
}

# Payment methods and their usage frequency
payment_methods = {
    'Credit Card': 0.60,
    'Debit Card': 0.25,
    'Cash': 0.10,
    'Online Transfer': 0.05
}

def generate_expense_data(start_date, end_date, transactions_per_day_avg=3):
    """Generate realistic expense data"""
    
    expenses = []
    current_date = start_date
    
    while current_date <= end_date:
        # Weekend vs weekday patterns
        is_weekend = current_date.weekday() >= 5
        
        # Adjust transaction frequency
        if is_weekend:
            daily_transactions = np.random.poisson(transactions_per_day_avg * 0.7)
        else:
            daily_transactions = np.random.poisson(transactions_per_day_avg)
        
        # Generate transactions for this day
        for _ in range(max(0, daily_transactions)):
            # Choose category based on frequency weights
            category = np.random.choice(
                list(expense_categories.keys()),
                p=[cat['frequency'] for cat in expense_categories.values()]
            )
            
            # Generate amount within category range
            cat_info = expense_categories[category]
            amount = round(np.random.uniform(cat_info['min'], cat_info['max']), 2)
            
            # Choose payment method
            payment_method = np.random.choice(
                list(payment_methods.keys()),
                p=list(payment_methods.values())
            )
            
            # Generate description
            descriptions = {
                'Groceries': ['SUPERMARKET', 'GROCERY STORE', 'WHOLE FOODS', 'TRADER JOES', 'COSTCO'],
                'Restaurants': ['RESTAURANT', 'CAFE', 'PIZZA HUT', 'MCDONALDS', 'STARBUCKS'],
                'Gas': ['SHELL', 'CHEVRON', 'BP GAS', 'EXXON', 'ARCO'],
                'Shopping': ['AMAZON', 'TARGET', 'WALMART', 'BEST BUY', 'MACY\'S'],
                'Entertainment': ['NETFLIX', 'SPOTIFY', 'MOVIE THEATER', 'GAMING', 'CONCERT'],
                'Bills_Utilities': ['ELECTRIC BILL', 'WATER BILL', 'INTERNET', 'PHONE BILL', 'RENT'],
                'Transportation': ['UBER', 'LYFT', 'BUS FARE', 'METRO', 'PARKING'],
                'Healthcare': ['PHARMACY', 'DOCTOR VISIT', 'DENTIST', 'HOSPITAL', 'PRESCRIPTION'],
                'Personal_Care': ['HAIRCUT', 'SALON', 'GYM', 'SPA', 'BEAUTY SUPPLY'],
                'Miscellaneous': ['ATM FEE', 'BANK FEE', 'GIFT', 'DONATION', 'OTHER']
            }
            
            description = random.choice(descriptions[category])
            
            # Add some transaction ID for realism
            transaction_id = f"TXN{random.randint(100000, 999999)}"
            
            expenses.append({
                'transaction_id': transaction_id,
                'date': current_date.strftime('%Y-%m-%d'),
                'amount': -amount,  # Expenses are negative
                'category': category,
                'description': description,
                'payment_method': payment_method,
                'merchant': description
            })
        
        current_date += timedelta(days=1)
    
    return expenses

# Generate 12 months of data
start_date = datetime(2023, 8, 1)  # Start from August 2023
end_date = datetime(2024, 7, 31)   # End at July 2024

print("Generating expense data...")
sample_expenses = generate_expense_data(start_date, end_date)

# Convert to DataFrame
df = pd.DataFrame(sample_expenses)

# Add some income entries (monthly salary)
income_entries = []
current_month = start_date.replace(day=1)
while current_month <= end_date:
    # Add salary on 1st of each month
    salary_date = current_month.strftime('%Y-%m-%d')
    income_entries.append({
        'transaction_id': f"SAL{random.randint(100000, 999999)}",
        'date': salary_date,
        'amount': 4500.00,  # Monthly salary
        'category': 'Salary',
        'description': 'MONTHLY SALARY',
        'payment_method': 'Direct Deposit',
        'merchant': 'EMPLOYER NAME'
    })
    
    # Move to next month
    if current_month.month == 12:
        current_month = current_month.replace(year=current_month.year + 1, month=1)
    else:
        current_month = current_month.replace(month=current_month.month + 1)

# Add income to expenses
df_income = pd.DataFrame(income_entries)
df_complete = pd.concat([df, df_income], ignore_index=True)

# Sort by date
df_complete['date'] = pd.to_datetime(df_complete['date'])
df_complete = df_complete.sort_values('date').reset_index(drop=True)
df_complete['date'] = df_complete['date'].dt.strftime('%Y-%m-%d')

print(f"Generated {len(df_complete)} transactions")
print(f"Date range: {df_complete['date'].min()} to {df_complete['date'].max()}")

# Save to CSV
df_complete.to_csv('../data/raw/sample_expenses_2023_2024.csv', index=False)

print("✅ Sample data saved to data/raw/sample_expenses_2023_2024.csv")

# Display summary
print(f"\nData Summary:")
print(f"Total transactions: {len(df_complete)}")
print(f"Date range: {df_complete['date'].min()} to {df_complete['date'].max()}")
print(f"\nCategory breakdown:")
print(df_complete['category'].value_counts())
print(f"\nPayment method breakdown:")
print(df_complete['payment_method'].value_counts())

# Show first few rows
df_complete.head(10)